# <center> Panongbene Jean Mohamed Sawadogo </center>#

* $\color{red}{Course}$        : Visual Analytics, CentraleSupelec
* $\color{red}{Professor}$     : Petra Isenberg, Wesley Willett
* $\color{red}{Assignement}$   : second Assignement
* $\color{red}{Author}$        : Panongbene Jean Mohamed Sawadogo
* $\color{red}{Title}$         : Collecting & Cleaning Data

$\color{red}{Project - description : }$  In this project, we scrappons the site https://www.populationdata.net/pays/ to extract all the information relating to the countries of the world at the end of constituting a database of countries with the maximum information that we can have on each country.



# We import the necessary libraries

In [470]:
import pandas as pd
from bs4 import BeautifulSoup

import os.path
from os import path
import requests 

# All Extras Parameters

In [471]:
#lin_web_page_baudlaire = 'http://www.toutelapoesie.com/poemes/baudelaire/'
#link_web_page_apollinaire = 'http://www.toutelapoesie.com/poemes/apollinaire/'

In [472]:
link_web_page = 'https://www.populationdata.net/pays/'

# the real name in the web page of datafram column
array_column = ['Nom officiel','Nom propre','Continent','Sous-continent','Population',
                'Croissance démographique','Superficie','Densité','PIB','PIB/habitant',
                'Croissance du PIB','Espérance de vie',
                'Taux de natalité','Indice de fécondité','Taux de mortalité','Taux de mortalité infantile(‰ (2019))',
                'Taux d\'alphabétisation','Langues officielles','Monnaie','IDH','IPE','Nature de l"État','Chef de l"État',
                'Fête nationale','Codes ISO','Gentilé','Touristes','Site WikiPedia Page']

# the names of all datafram column
datafram_column = ['Nom officiel','Nom propre','Continent','Sous-continent','Population(habitants (2020))',
                  'Croissance démographique(% / an)','Superficie(km²)','Densité(habitants / km²)',
                  'PIB(milliards $USD (2018))','PIB/habitant','Croissance du PIB(% / an (2018))',
                  'Espérance de vie(ans (2019))','Taux de natalité(‰ (2019))','Indice de fécondité(enfants / femme (2019))',
                  'Taux de mortalité(‰ (2019))','Taux de mortalité infantile(‰ (2019))','Taux d"alphabétisation(% (2020))',
                   'Langues officielles','Monnaie','IDH(/ 1 (2018))','IPE','Nature de l"État','Chef de l"État','Fête nationale',
                  'Codes ISO','Gentilé','Touristes','Site WikiPedia Page']

# all caractere that we use for cleaning
array_used_clean = [" ("," habitants (2018)"," % / an"," km²"," habitants / km²"," milliards $USD (2017)",
                    " $USD (2017)"," % / an (2017)"," ans (2018)"," habitants /"," personnes (2017)",
                    " personnes"," $USD"," habitants","milliards $USD"]

In [473]:
# add a new line in ours datafram
def add_new_line_data_fram(name, dictionnary_data_new,datafram_country_data):
    """Uses this function for to add a new line in ours data frame. The new line values are 
    the values in dictionnary_data_new"""
    
    # We create the new line in data Fram
    datafram_country_data.loc[name] = 'NAN'
    
    # We put the values in ours new line
    j = 0
    for i in dictionnary_data_new:
        datafram_country_data.loc[name][j] = dictionnary_data_new[i]
        j+=1
        

def clean_the_data_frame_data(values_string,array_used_clean):
    """We use this function for clean the values that we will put in the datafram"""
    for i in array_used_clean:
        kkk = values_string.split(i)
        values_string = kkk[0]

    return values_string.replace("n.c.","NaN")


## TRY IT
Write a function to fetch and save all of the vispubdata pages found here: 


In [474]:
#function 
def fetch_scrapping_pages():
    """We use this function to scrapes all informations about each country. This function
     return a pandas datafram which contains all informations of each countries"""
    # page Request
    page_country_first = requests.get(link_web_page, allow_redirects=True)
    
    #we process the request
    if not page_country_first.status_code == 200:
        print(" ERROR Status code: " + str(page_country_first.status_code))
        return
    else:
        soup = BeautifulSoup(page_country_first.content)
        
        # we create the data frame
        datafram_country_data = pd.DataFrame(columns=datafram_column)
    
        for tablecountry in soup.findAll('table'):
            for onecountry in tablecountry.findAll('tr'):
                # name country
                name_country = ''
                # we create the dictionnary
                dictionnary_data = {}
                
                if('data-iso-code' in onecountry.attrs):
                    for country in onecountry.findAll('a'):
                        if('pays' in country['href']):
                            link_country = country['href']
                            name_country = country.text
                            
                            # page Request
                            page_country = requests.get(link_country, allow_redirects=True)
                            #we process the request
                            if not page_country.status_code == 200:
                                print(" ERROR Status code: " + str(page_country.status_code))
                                continue
                            else:
                                soup_page_country = BeautifulSoup(page_country.content)
                                
                                for tablecountry in soup_page_country.findAll('tbody'):
                                    for table_infos in tablecountry.findAll('tr'):
                                        keys_values = []
                                        for information_line in table_infos.findAll('td'):
                                            keys_values.append(information_line.text)
                                       
                                        keys_values[0] = clean_the_data_frame_data(keys_values[0],array_used_clean)
                                        keys_values[1] = clean_the_data_frame_data(keys_values[1],array_used_clean)
                                        dictionnary_data[keys_values[0]] = keys_values[1]
                                    #we use break here because some page have a lot of tbody
                                    break
                                # we add the new ligne in the datafram
                                add_new_line_data_fram(name_country,dictionnary_data,datafram_country_data)
    return datafram_country_data

In [475]:
#<tr data-iso-code=""><td><a name="Abkhazie" href="https://www.populationdata.net/pays/abkhazie/">Abkhazie</a></td><td><a href="https://www.populationdata.net/continents/asie/">Asie</a></td><td><a href="https://www.populationdata.net/sous-continents/asie-de-louest/">Asie de l'Ouest</a></td></tr>
datafram_country_data = fetch_scrapping_pages()
datafram_country_data.head()

,Nom officiel,Nom propre,Continent,Sous-continent,Population(habitants (2020)),Croissance démographique(% / an),Superficie(km²),Densité(habitants / km²),PIB(milliards $USD (2018)),PIB/habitant,...,Monnaie,IDH(/ 1 (2018)),IPE,"Nature de l""État","Chef de l""État",Fête nationale,Codes ISO,Gentilé,Touristes,Site WikiPedia Page
Abkhazie,République d'Abkhazie,Аҧсны Аҳәынҭқарра,Asie,Asie de l'Ouest,246 313,"0,49",8 653,"28,47","0,336 milliards",1 372,...,"Apsar abkhaze, Rouble russe",NaN,NaN,République,Président Raul Khadjimba,23 juillet,",","Abkhaze, Abkhazienne, Abkhazien",1 000 000,NAN
Açores,Région autonome des Açores,Região Autónoma dos Açores,Europe,Union Européenne,243 862,"-0,39",2 322,"105,02","4,220 milliards",17 398,...,Euro,NaN,NaN,République,Président Marcelo Rebelo de Sousa,10 juin,",","Açoréenne, Açoréen, Portugaise, Portugais",1 487 428,NAN
Afghanistan,République islamique d'Afghanistan,د أفغانستان اسلامي جمهوریت,Asie,Asie centrale,34 233 212,"6,23",652 864,"52,44","19,363 milliards",521,...,Afghani,"0,496 / 1","37,74",République islamique,Président Ashraf Ghani,19 août,"AF, AFG",Afghan,20 000,NAN
Afrique du Sud,République d'Afrique du Sud,Republiek van Suid-Afrika,Afrique,Afrique Australe,57 725 606,"2,13",1 219 912,"47,32","366,298 milliards",6 340,...,Rand,"0,705 / 1","44,73","République unitaire parlementaire, régime prés...",Président Cyril Ramaphosa,27 avril,"ZA, ZAF","Sud-Africaine, Sud-Africain",10 285 000,NAN
Albanie,République d'Albanie,Republika e Shqipërisë,Europe,Europe,2 870 324,"-0,22",28 748,"99,84","15,059 milliards",5 254,...,Lek,"0,791 / 1","65,46",République,Président Ilir Meta,28 novembre,"AL, ALB","Albanaise, Albanais",5 375 174,NAN


In [476]:
print(datafram_country_data.head())

                                      Nom officiel  \
Abkhazie                     République d'Abkhazie   
Açores                  Région autonome des Açores   
Afghanistan     République islamique d'Afghanistan   
Afrique du Sud         République d'Afrique du Sud   
Albanie                       République d'Albanie   

                                Nom propre Continent    Sous-continent  \
Abkhazie                 Аҧсны Аҳәынҭқарра      Asie   Asie de l'Ouest   
Açores          Região Autónoma dos Açores    Europe  Union Européenne   
Afghanistan     د أفغانستان اسلامي جمهوریت      Asie     Asie centrale   
Afrique du Sud   Republiek van Suid-Afrika   Afrique  Afrique Australe   
Albanie             Republika e Shqipërisë    Europe            Europe   

               Population(habitants (2020)) Croissance démographique(% / an)  \
Abkhazie                            246 313                             0,49   
Açores                              243 862                           

Write a function that processes all of the saved vispubdata files and produces a CSV


In [140]:
def process_vispubdata_pages():
    
    for file_name in name_file_visite_pub:
        with open(file_name, encoding='utf-8') as fp:
            soup = BeautifulSoup(fp, 'html.parser')
            for div in soup.findAll('div'):
                print(div['class'][0])
                print(div.text)
                #print()
    # Save the CSV File 
    #csv_file_name = directory_visite_pub+"/visite_pub_information.csv"
    #df.to_csv(csv_file_name)
    #print("Saved " +csv_file_name)
    
    
    #TODO: Process vispubdata pages
    #print('TODO: Process vispubdata pages to a CSV')


In [106]:
print('é')

é


In [1]:
#fetch_vispubdata_pages()
#process_vispubdata_pages()